In [ ]:
##################### implementation of info GAN

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import itertools
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F
import torchvision.utils as vutils

In [ ]:
batch_size=100
# transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(mean=(0.5, ), std=(0.5, ))])
transform = transforms.Compose([
                                transforms.ToTensor()
])
train_data = torchvision.datasets.MNIST('/content/', train = True,
                           transform=transform, download = True)
test_data = torchvision.datasets.MNIST('/content/', train = False,
                           transform=transform, download = True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
class Generator(nn.Module):
    def __init__(self, nz=74):
      super(Generator, self).__init__()
      self.l1 = nn.Linear(74, 1024, bias=False)
      # self.conv1 = nn.ConvTranspose2d(nz,1024,1,1, bias=False)
      self.b1 = nn.BatchNorm1d(1024)

      self.l2 = nn.Linear(1024,7*7*128, bias=False)
      # self.conv2 = nn.ConvTranspose2d(1024,128,7,1, bias=False)
      self.b2 = nn.BatchNorm2d(128)

      self.up1 = nn.ConvTranspose2d(128,64,4,stride=2,padding=1, bias=False)
      self.b3 = nn.BatchNorm2d(64)
      
      self.up2 = nn.ConvTranspose2d(64,1,4,stride=2,padding=1, bias=False)
  
    def forward(self, input):
      output = self.b1(F.relu(self.l1(input)))
      # output = F.relu(output)
      # output = self.b1(output)
      output = self.l2(output)
      output = output.view(-1,128,7,7)
      output = self.b2(F.relu(output))
      output = self.b3(F.relu(self.up1(output)))
      output = torch.sigmoid(self.up2(output))
      # print(output.shape)
      return output

class DQ_common(nn.Module):
    def __init__(self):
      super(DQ_common, self).__init__()
      self.conv1 = nn.Conv2d(1,64,4,stride=2,padding=1, bias=False)

      self.conv2 = nn.Conv2d(64,128,4,stride=2,padding=1, bias=False)
      self.b2 = nn.BatchNorm2d(128)

      self.l1 = nn.Linear(7*7*128,1024)
      self.b3 = nn.BatchNorm1d(1024)
      # self.conv3 = nn.Conv2d(128, 1024, 7, bias=False)
      # self.b3 = nn.BatchNorm2d(1024)

    def forward(self,input):
      output = F.leaky_relu(self.conv1(input), 0.1, inplace=True)
      output = F.leaky_relu(self.conv2(output), 0.1, inplace=True)
      output = self.b2(output)
      output = output.view(-1,128*7*7)
      # output = torch.flatten(output)
      # print(output.shape)
      output = self.b3(F.leaky_relu(self.l1(output), 0.1, inplace=True))
      # print(output.shape)
      return output

class Dis(nn.Module):
    def __init__(self):
      super(Dis, self).__init__()
      self.l1 = nn.Linear(1024,1)
      # self.conv1 = nn.Conv2d(1024,1,1, bias=False)
      
    def forward(self,input):
      output = torch.sigmoid(self.l1(input)).squeeze() #.view(-1,1)
      return output

class Qhead(nn.Module):
    def __init__(self):
      super(Qhead,self).__init__()
      self.l1 = nn.Linear(1024,128)
      # self.conv1 = nn.Conv2d(1024,128,1, bias=False)
      self.b1 = nn.BatchNorm1d(128)
      self.l2 = nn.Linear(128,12)
      # self.conv2 = nn.Conv2d(128,10,1, bias=False)
      # self.conv3 = nn.Conv2d(128,2,1, bias=False)
      # self.conv4 = nn.Conv2d(128,2,1, bias=False)
      
    def forward(self, input):
      out = F.leaky_relu(self.b1(self.l1(input)), 0.1, inplace=True)
      out = self.l2(out)

      # discrete = self.conv2(out).squeeze()
      # mu = self.conv3(out).squeeze()
      # var = torch.exp(self.conv4(out).squeeze())

      # return discrete, mu, var
      return out



In [ ]:
import random
seed=1123
random.seed(seed)
torch.manual_seed(seed)

In [ ]:
def weights_init(m):
    if(type(m) == nn.ConvTranspose2d or type(m) == nn.Conv2d):
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif(type(m) == nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def noise_function(noise, con_c, dis_c, bs,device):

    idx = np.random.randint(10, size=bs)
    c = np.zeros((bs, 10))
    c[range(bs),idx] = 1.0

    dis_c.copy_(torch.Tensor(c))
    # con_c.data.uniform_(-1.0, 1.0)
    # noise.data.uniform_(-1.0, 1.0)
    z = torch.cat([noise, dis_c, con_c], dim=1) #.view(-1, 74, 1, 1)
    # z=z.to(device)
    idx = torch.from_numpy(idx).to(device)
    return z, idx



def log_gaussian(x, mu, var):
  logli = -0.5*(var.mul(2*np.pi)+1e-6).log() - (x-mu).pow(2).div(var.mul(2.0)+1e-6)

  return logli.sum(1).mean().mul(-1)

In [ ]:
device='cuda'
epochs=80
nz=74
# fixed_noise = torch.randn((16,nz,1,1),dtype=torch.float32,device=device)
netGenerator = Generator().to(device)
netGenerator.apply(weights_init)
netDQ_common = DQ_common().to(device)
netDQ_common.apply(weights_init)
netDis = Dis().to(device)
netDis.apply(weights_init)
netQhead = Qhead().to(device)
netQhead.apply(weights_init)

criterion = nn.BCEWithLogitsLoss() #nn.BCELoss()
criterionQ_dis = nn.CrossEntropyLoss()
# criterionQ_con = log_gaussian
criterionQ_con = nn.MSELoss()

real_label = torch.full((batch_size,),1,dtype=torch.float32,device=device)
fake_label = torch.full((batch_size,),0,dtype=torch.float32,device=device)
optimizer_g = optim.Adam([{'params': netGenerator.parameters()}, {'params': netQhead.parameters()}], lr=0.001)
optimizer_d = optim.Adam([{'params': netDQ_common.parameters()}, {'params': netDis.parameters()}], lr=0.0002)

In [ ]:
z = torch.randn(100, 62, 1, 1, device=device)
fixed_noise = z
idx = np.arange(10).repeat(10)
dis_c = torch.zeros(100, 1, 10, device=device)
for i in range(1):
    dis_c[torch.arange(0, 100), i, idx] = 1.0

dis_c = dis_c.view(100, -1, 1, 1)

con_c = torch.rand(100, 2, 1, 1, device=device) * 2 - 1
fixed_noise = torch.cat((fixed_noise, dis_c, con_c), dim=1).squeeze()
# fixed_noise.shape

In [ ]:
for epoch in range(epochs):
  Dis_loss=0
  G_loss=0
  Discrete_loss=0
  Cont_loss=0
  for i, (data,_) in enumerate(train_loader):
    data = data.to(device)
    bs = data.size(0)
    noise = torch.FloatTensor(bs,62).uniform_(-1.0,1.0).to(device)
    cont = torch.FloatTensor(bs,2).uniform_(-1.0,1.0).to(device)
    dis_c = torch.FloatTensor(bs,10).to(device)
    z, id = noise_function(noise,cont,dis_c,bs,device)
    ##discriminator
    optimizer_d.zero_grad()
    comm1 = netDQ_common(data)
    D_x = netDis(comm1)
    real_loss = criterion(D_x,real_label)
    real_loss.backward()

    # with torch.no_grad():
    # print(z.shape)
    G_z = netGenerator(z)
    comm2 = netDQ_common(G_z.detach())
    DG_z = netDis(comm2)
    fake_loss = criterion(DG_z,fake_label)
    fake_loss.backward()
    dis_loss = (real_loss+fake_loss)
    # dis_loss.backward()
    optimizer_d.step()
    Dis_loss+=(dis_loss.item())

    #generator
    optimizer_g.zero_grad()
    # gen_z = netGenerator(z)
    comm3 = netDQ_common(G_z)
    Dgen_z = netDis(comm3)
    g_loss = criterion(Dgen_z, real_label)
    G_loss+=(g_loss.item())
    #Q
    pred_c = netQhead(comm3)
    discrete_loss = criterionQ_dis(pred_c[:,:10],id)
    con_loss = criterionQ_con(z[:,-2:], pred_c[:,10:])*0.1 #criterionQ_con(z[:,-2:], pred_c[:,10:], pred_c[:,10:])*0.1
    Discrete_loss+=(discrete_loss.item())
    Cont_loss+=(con_loss.item())

    gen_loss = g_loss + discrete_loss + con_loss
    gen_loss.backward()
    optimizer_g.step()

    # if i%300==0:
      

  print('Epoch{}/{}: Discriminator loss:{:.6f}, Generator loss:{:.6f}, Discrete loss:{:.6f}, Continious loss:{:.6f}'.format(
      epoch+1, epochs, Dis_loss/(bs*len(train_loader)), G_loss/(bs*len(train_loader)), Discrete_loss/(bs*len(train_loader)), Cont_loss/(bs*len(train_loader)) ))
  # with torch.no_grad():
  #   sample = netGenerator(z)
  path = '/content/drive/MyDrive/VAE/InfoGAN/' + str(epoch+1) + '.png'
  # save_image(sample, path, nrow=10)
  print('Testing on Fixed data')
  with torch.no_grad():
    gen_data = netGenerator(fixed_noise).detach().cpu()
  plt.figure(figsize=(10, 10))
  plt.axis("off")
  plt.imshow(np.transpose(vutils.make_grid(gen_data, nrow=10, padding=2, normalize=True), (1,2,0)))
  plt.savefig(path)
